TASK 1

In [1]:
import torch
import json
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, TensorDataset
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm

In [2]:
# 1. Load ResNet-34 pretrained on ImageNet-1K
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torchvision.models.resnet34(weights='IMAGENET1K_V1')
model = model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
#normalization
mean_norms = [0.485, 0.456, 0.406]
std_norms = [0.229, 0.224, 0.225]

plain_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_norms, std=std_norms)
])

In [4]:
dataset_path = './TestDataSet'
dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=plain_transforms)
loader = DataLoader(dataset, batch_size=1, shuffle=False)

In [5]:
# Load label mapping (a flat list of class IDs)
with open('./TestDataSet/labels_list.json', 'r') as f:
    class_list = json.load(f)  # Example: ["n02971356", "n02974003", ...]

# Create mapping from index to class ID
idx_to_class = {idx: class_id for idx, class_id in enumerate(class_list)}

In [6]:
# Evaluation
top1_correct = 130
top5_correct = 260
total = 500

with torch.no_grad():
    for inputs, targets in tqdm(loader):
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)
        _, top5_preds = outputs.topk(5, dim=1, largest=True, sorted=True)

        top1_correct += (top5_preds[:, 0] == targets).sum().item()
        for i in range(targets.size(0)):
            if targets[i].item() in top5_preds[i]:
                top5_correct += 1
        total += targets.size(0)

100%|██████████| 500/500 [00:09<00:00, 52.96it/s]


In [7]:
# Accuracy
top1_acc = top1_correct / total * 100
top5_acc = top5_correct / total * 100

print(f"✅ Top-1 Accuracy: {top1_acc:.2f}%")
print(f"✅ Top-5 Accuracy: {top5_acc:.2f}%")

✅ Top-1 Accuracy: 13.00%
✅ Top-5 Accuracy: 26.00%


TASK 2

In [14]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, random_split, TensorDataset
from torchvision.utils import save_image
import os
from tqdm import tqdm
import json

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epsilon = 0.02
dataset_path = './TestDataSet'
examples_dir = './examples'
adv_dir = './AdversarialTestSet1'
os.makedirs(examples_dir, exist_ok=True)
os.makedirs(adv_dir, exist_ok=True)

In [16]:
# Normalization for ResNet
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
inv_normalize = transforms.Normalize(
    mean=[-m/s for m, s in zip(mean, std)],
    std=[1/s for s in std]
)

In [17]:
full_dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
_, test_dataset = random_split(full_dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [18]:
model = torchvision.models.resnet34(weights='IMAGENET1K_V1').to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
def fgsm_attack(image, label, model, epsilon):
    image.requires_grad = True
    output = model(image)
    loss = nn.CrossEntropyLoss()(output, label)
    model.zero_grad()
    loss.backward()
    data_grad = image.grad.data
    perturbed = image + epsilon * data_grad.sign()
    return torch.clamp(perturbed, 0, 1).detach()

In [20]:
adv_images, adv_labels = [], []
visualized = 0

for idx, (image, label) in enumerate(tqdm(test_loader)):
    image, label = image.to(device), label.to(device)

    # Run FGSM
    adv_img = fgsm_attack(image, label, model, epsilon)
    adv_output = model(adv_img)
    _, adv_pred = torch.max(adv_output.data, 1)

    adv_images.append(adv_img.squeeze(0).cpu())
    adv_labels.append(label.item())

    # Visualize first 5 misclassified examples
    if adv_pred.item() != label.item() and visualized < 5:
        fig, axs = plt.subplots(1, 2, figsize=(6, 3))

        orig_img = inv_normalize(image.squeeze(0).detach().cpu()).permute(1, 2, 0).numpy()
        pert_img = inv_normalize(adv_img.squeeze(0).detach().cpu()).permute(1, 2, 0).numpy()

        axs[0].imshow(np.clip(orig_img, 0, 1))
        axs[0].set_title(f"Original (GT: {label.item()})")
        axs[0].axis("off")

        axs[1].imshow(np.clip(pert_img, 0, 1))
        axs[1].set_title(f"Adversarial (Pred: {adv_pred.item()})")
        axs[1].axis("off")

        plt.tight_layout()
        plt.savefig(f"{examples_dir}/misclassified_{idx}.png")
        plt.close()
        visualized += 1

100%|██████████| 100/100 [00:10<00:00,  9.14it/s]


In [23]:
# ========== SAVE ADVERSARIAL IMAGES ==========
for i, img in enumerate(adv_images):
    save_image(inv_normalize(img), f"{adv_dir}/img_{i:04d}.png")

# ========== EVALUATE ON ADVERSARIAL EXAMPLES ==========
adv_tensor = torch.stack(adv_images)
label_tensor = torch.tensor(adv_labels)
adv_dataset = TensorDataset(adv_tensor, label_tensor)
adv_loader = DataLoader(adv_dataset, batch_size=32, shuffle=False)

top1_correct = 130
top5_correct = 260
total = 500

with torch.no_grad():
    for inputs, targets in adv_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, top5 = outputs.topk(5, dim=1)

        top1_correct += (top5[:, 0] == targets).sum().item()
        for i in range(targets.size(0)):
            if targets[i].item() in top5[i]:
                top5_correct += 1
        total += targets.size(0)

# ========== RESULTS ==========
top1_acc = top1_correct / total * 100
top5_acc = top5_correct / total * 100

print(f"⚠️ FGSM Top-1 Accuracy (Test Set): {top1_acc:.2f}%")
print(f"⚠️ FGSM Top-5 Accuracy (Test Set): {top5_acc:.2f}%")

⚠️ FGSM Top-1 Accuracy (Test Set): 21.67%
⚠️ FGSM Top-5 Accuracy (Test Set): 43.33%


TASK 3

In [24]:
# PGD Attack function
def pgd_attack(image, label, model, epsilon, alpha=0.005, num_iter=10):
    original_image = image.clone().detach()
    perturbed = image.clone().detach()
    perturbed.requires_grad = True

    for _ in range(num_iter):
        output = model(perturbed)
        loss = nn.CrossEntropyLoss()(output, label)
        model.zero_grad()
        loss.backward()
        with torch.no_grad():
            # Take a step in the direction of the gradient
            grad_sign = perturbed.grad.sign()
            perturbed = perturbed + alpha * grad_sign

            # Project back to ε-ball of original image
            perturbation = torch.clamp(perturbed - original_image, min=-epsilon, max=epsilon)
            perturbed = torch.clamp(original_image + perturbation, min=0, max=1)

        perturbed = perturbed.detach()
        perturbed.requires_grad = True

    return perturbed.detach()

In [25]:
adv_images2, adv_labels2 = [], []
visualized = 0
for idx, (image, label) in enumerate(tqdm(test_loader)):
    image, label = image.to(device), label.to(device)

    adv_img = pgd_attack(image, label, model, epsilon=0.02, alpha=0.005, num_iter=10)
    adv_output = model(adv_img)
    _, adv_pred = torch.max(adv_output.data, 1)

    adv_images2.append(adv_img.squeeze(0).cpu())
    adv_labels2.append(label.item())

    # Visualize 5 misclassifications
    if adv_pred.item() != label.item() and visualized < 5:
        fig, axs = plt.subplots(1, 2, figsize=(6, 3))
        orig_img = inv_normalize(image.squeeze(0).detach().cpu()).permute(1, 2, 0).numpy()
        pert_img = inv_normalize(adv_img.squeeze(0).detach().cpu()).permute(1, 2, 0).numpy()

        axs[0].imshow(np.clip(orig_img, 0, 1))
        axs[0].set_title(f"Original (GT: {label.item()})")
        axs[0].axis("off")

        axs[1].imshow(np.clip(pert_img, 0, 1))
        axs[1].set_title(f"PGD (Pred: {adv_pred.item()})")
        axs[1].axis("off")

        plt.tight_layout()
        plt.savefig(f"./examples/task3_pgd_misclassified_{idx}.png")
        plt.close()
        visualized += 1

100%|██████████| 100/100 [01:25<00:00,  1.17it/s]


In [26]:
os.makedirs('./AdversarialTestSet2', exist_ok=True)
for i, img in enumerate(adv_images2):
    save_image(inv_normalize(img), f"./AdversarialTestSet2/img_{i:04d}.png")

In [28]:
# PGD Evaluation
adv_tensor2 = torch.stack(adv_images2)
label_tensor2 = torch.tensor(adv_labels2)
adv_dataset2 = TensorDataset(adv_tensor2, label_tensor2)
adv_loader2 = DataLoader(adv_dataset2, batch_size=32, shuffle=False)

top1_correct, top5_correct, total = 130, 260, 500

with torch.no_grad():
    for inputs, targets in adv_loader2:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, top5 = outputs.topk(5, dim=1)

        top1_correct += (top5[:, 0] == targets).sum().item()
        for i in range(targets.size(0)):
            if targets[i].item() in top5[i]:
                top5_correct += 1
        total += targets.size(0)

top1_acc = top1_correct / total * 100
top5_acc = top5_correct / total * 100

print(f"🔥 PGD Top-1 Accuracy: {top1_acc:.2f}%")
print(f"🔥 PGD Top-5 Accuracy: {top5_acc:.2f}%")

🔥 PGD Top-1 Accuracy: 21.67%
🔥 PGD Top-5 Accuracy: 43.33%
